In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:80% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

In [7]:
import os # 모델 저장할 폴더가 있는지 여부와 폴더 생성
import numpy as np
import pandas as pd # get_dummies(원핫인코딩), crosstab
import seaborn as sns # iris 데이터(데이터프레임) 가져오기
from sklearn import datasets # iris 데이터(X, y) 가져오기
from sklearn.preprocessing import LabelEncoder # 라벨인코더
from tensorflow.keras.utils import to_categorical # 원핫인코딩 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras import metrics # compile
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

# 1. 기본적인 DNN

In [12]:
# 1. 데이터 생성 및 전처리
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                                   test_size=0.2,
                                                   stratify=iris_y)
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [18]:
# 2. 모델구성
model = Sequential([
    Input(4),
    Dense(units=64, activation='relu'),
    Dense(units=128, activation='relu'),
    Dropout(0.2), 
    Dense(units=50, activation='relu'),
    Dropout(0.2),
    Dense(units=30, activation='relu'),
    Dropout(0.1),
    Dense(units=3, activation='softmax')    
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                320       
                                                                 
 dense_6 (Dense)             (None, 128)               8320      
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 50)                6450      
                                                                 
 dropout_4 (Dropout)         (None, 50)                0         
                                                                 
 dense_8 (Dense)             (None, 30)                1530      
                                                                 
 dropout_5 (Dropout)         (None, 30)               

In [20]:
# 3. 학습과정 설정
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
# 4. 모델 학습
earlystopping = EarlyStopping(patience=40)# monitor='val_loss' 기본값
hist=model.fit(train_X,train_y,
               epochs=200,
               validation_split=0.1,
               callbacks=[earlystopping])

Epoch 1/200
4/4 [==============================] - 1s 65ms/step - loss: 1.2184 - accuracy: 0.3148 - val_loss: 1.0554 - val_accuracy: 0.6667
Epoch 2/200
4/4 [==============================] - 0s 11ms/step - loss: 1.1007 - accuracy: 0.3796 - val_loss: 0.9740 - val_accuracy: 0.9167
Epoch 3/200
4/4 [==============================] - 0s 10ms/step - loss: 1.0038 - accuracy: 0.5463 - val_loss: 0.8995 - val_accuracy: 0.6667
Epoch 4/200
4/4 [==============================] - 0s 10ms/step - loss: 0.9494 - accuracy: 0.6667 - val_loss: 0.8468 - val_accuracy: 0.6667
Epoch 5/200
4/4 [==============================] - 0s 10ms/step - loss: 0.8656 - accuracy: 0.6204 - val_loss: 0.8047 - val_accuracy: 0.6667
Epoch 6/200
4/4 [==============================] - 0s 10ms/step - loss: 0.8476 - accuracy: 0.6296 - val_loss: 0.7805 - val_accuracy: 0.6667
Epoch 7/200
4/4 [==============================] - 0s 10ms/step - loss: 0.7628 - accuracy: 0.6667 - val_loss: 0.7217 - val_accuracy: 0.6667
Epoch 8/200
4/4 [===

4/4 [==============================] - 0s 10ms/step - loss: 0.1440 - accuracy: 0.9352 - val_loss: 0.0532 - val_accuracy: 1.0000
Epoch 60/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1390 - accuracy: 0.9352 - val_loss: 0.0411 - val_accuracy: 1.0000
Epoch 61/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1394 - accuracy: 0.9444 - val_loss: 0.0397 - val_accuracy: 1.0000
Epoch 62/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1425 - accuracy: 0.9444 - val_loss: 0.0884 - val_accuracy: 0.9167
Epoch 63/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1213 - accuracy: 0.9444 - val_loss: 0.0403 - val_accuracy: 1.0000
Epoch 64/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1684 - accuracy: 0.9352 - val_loss: 0.0357 - val_accuracy: 1.0000
Epoch 65/200
4/4 [==============================] - 0s 9ms/step - loss: 0.1152 - accuracy: 0.9444 - val_loss: 0.0493 - val_accuracy: 1.0000
Epoch 66/200
4/4 [=========

4/4 [==============================] - 0s 9ms/step - loss: 0.0830 - accuracy: 0.9630 - val_loss: 0.0158 - val_accuracy: 1.0000
Epoch 118/200
4/4 [==============================] - 0s 13ms/step - loss: 0.0894 - accuracy: 0.9630 - val_loss: 0.0452 - val_accuracy: 1.0000
Epoch 119/200
4/4 [==============================] - 0s 9ms/step - loss: 0.1220 - accuracy: 0.9537 - val_loss: 0.0306 - val_accuracy: 1.0000
Epoch 120/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0660 - accuracy: 0.9815 - val_loss: 0.0155 - val_accuracy: 1.0000
Epoch 121/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1062 - accuracy: 0.9722 - val_loss: 0.0142 - val_accuracy: 1.0000
Epoch 122/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0900 - accuracy: 0.9630 - val_loss: 0.0267 - val_accuracy: 1.0000
Epoch 123/200
4/4 [==============================] - 0s 9ms/step - loss: 0.1283 - accuracy: 0.9630 - val_loss: 0.0265 - val_accuracy: 1.0000
Epoch 124/200
4/4 [======

Epoch 175/200
4/4 [==============================] - 0s 14ms/step - loss: 0.0938 - accuracy: 0.9630 - val_loss: 0.0445 - val_accuracy: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1025 - accuracy: 0.9444 - val_loss: 0.0297 - val_accuracy: 1.0000
Epoch 177/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0564 - accuracy: 0.9815 - val_loss: 0.0284 - val_accuracy: 1.0000
Epoch 178/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0572 - accuracy: 0.9722 - val_loss: 0.0183 - val_accuracy: 1.0000
Epoch 179/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0686 - accuracy: 0.9815 - val_loss: 0.0145 - val_accuracy: 1.0000
Epoch 180/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0763 - accuracy: 0.9722 - val_loss: 0.0131 - val_accuracy: 1.0000
Epoch 181/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0871 - accuracy: 0.9815 - val_loss: 0.0172 - val_accuracy: 1.0000
Epoch 1

# 2. sklearn 이용
- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동

In [22]:
from sklearn.neural_network import MLPClassifier

In [23]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [24]:
model = MLPClassifier(
            hidden_layer_sizes=(64, 128, 50), # hidden layer의 units 수
            activation='relu',
            solver-'adam', # sgd등의 optimizer
            batch_size=40,
            max_iter=1000, # 학습 최대 횟수
            early_stopping=True, # 조기 종료 활성화
            n_iter_no_change=10, # earlyStopping의 patience와 유사
            validation_fraction=0.1, # 검증셋 비율
            warm_start = False # True일 경우 이전학습에 이어서 학습
)

In [26]:

model.fit(train_X, train_y)

C:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(64, 128, 50))

In [27]:
iris_X[0]

array([5.1, 3.5, 1.4, 0.2])

In [28]:
# 모델 사용
input_data = [[5.1, 3.5, 1.4, 0.2]]
model.predict(input_data)

array([0])

In [29]:
# 교차표
# test_y : 실제값
hat_y = model.predict(test_X)
pd.crosstab(test_y, hat_y, rownames=['real'], colnames=['predict'])

predict,0,1,2
real,,,
0,10,0,0
1,0,10,0
2,0,1,9


# 3. 클래스를 생성하여 모델 생성함수 사용

In [32]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu'):
#         model = Sequential()
#         model.add(Input(input_dim))
#         model.add(Dense(units=50, activation=activation))
#         model.add(Dense(units=30, activation=activation))
#         model.add(Dense(units=3, activation='softmax'))
        # 모델구성
        model = Sequential([
            Input(input_dim),
            Dense(units=50, activation=activation),
            Dense(units=30, activation=activation),
            Dense(units=3, activation='softmax')
        ])
        # 학습 설정
        model.compile(loss='sparse_categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])
        return model      

In [40]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape
# 2. 모델
model = DNNClassifier.build(input_dim=4, activation='elu')
# 3.모델 학습
hist = model.fit(train_X, train_y, epochs=500, validation_split=0.1, verbose= 0)

In [41]:
# 모델 평가

loss, accuracy = model.evaluate(test_X, test_y)
print(f'정확도 : {accuracy*100} %')

1/1 [==============================] - 0s 19ms/step - loss: 0.0671 - accuracy: 0.9667
정확도 : 96.66666388511658 %


# 4. 함수형 API
- 병렬처리, Resicual block

In [51]:
# 기존 model 스타일1
model = Sequential()
model.add(Input(shape=(4,)))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))
# 기존 model 스타일2
model = Sequential([
    Input(shape=(4,)),
    Dense(units=50, activation='relu'),
    Dense(units=30, activation='relu'),
    Dense(units=3, activation='softmax')
])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 50)                250       
                                                                 
 dense_49 (Dense)            (None, 30)                1530      
                                                                 
 dense_50 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [58]:
# 기존 model 스타일3 (함수형 API)
from tensorflow.keras import Model
input_ = Input(shape=(4,))
layer1 = Dense(units=50, activation='relu')(input_)
layer2 = Dense(units=30, activation='relu')(layer1)
output = Dense(units=3, activation='softmax')(layer2)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 4)]               0         
                                                                 
 dense_60 (Dense)            (None, 50)                250       
                                                                 
 dense_61 (Dense)            (None, 30)                1530      
                                                                 
 dense_62 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [59]:
# 병렬처리 방식
from tensorflow.keras.layers import concatenate
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=80, activation='relu')(input_)
dense3 = Dense(units=30, activation='relu')(input_)
x = concatenate([dense1, dense2, dense3])
output = Dense(units=3, activation='softmax')(x)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_63 (Dense)               (None, 50)           250         ['input_23[0][0]']               
                                                                                                  
 dense_64 (Dense)               (None, 80)           400         ['input_23[0][0]']               
                                                                                                  
 dense_65 (Dense)               (None, 30)           150         ['input_23[0][0]']               
                                                                                            

In [60]:
# 리지듀얼 블얼(Residual Block)
    #딥러닝에서 기울기 소실 문제로 학습이 잘 되지 않는 부분을 해결하기 위한 제안
    
from tensorflow.keras.layers import add

input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=50, activation='relu')(dense1)
dense3 = add([dense1, dense2])
output = Dense(units=3, activaiton='softmax')(dense3)
model = Model(inputs=input_, outputs = output)

TypeError: ('Keyword argument not understood:', 'activaiton')

In [61]:
# 학습과정 설정 & 학습
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=100, verbose=0)

In [62]:
# 학습 평가
loss, accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f'accuracy : {accuracy}')

accuracy : 0.9666666388511658
